In [27]:
#Finding Mean crash rate 
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

# Load data
data = pd.read_csv("/Users/lebakuprathyushkumarreddy/Desktop/Prof_Lu_Gao/Crash_Data_(SOR)_2022_IRI_others.csv")
# Preprocess data
data["IRI"].fillna(df["IRI"].mean(), inplace=True)
data['Crash_Frequency'].fillna(df['Crash_Frequency'].mean(),inplace=True)

df = data[data["LIGHT"] == 1]    #subset of the data for different road conditions

iri_bins = [0, 95, 170, float('inf')]
iri_labels = ['Good', 'Fair', 'Poor']

df['IRI_Category'] = pd.cut(pd.to_numeric(df['IRI'], errors='coerce'), bins=iri_bins, labels=iri_labels)

# Calculate mean crash rates for each IRI category
mean_crash_rates = df.groupby('IRI_Category')['Crash_Frequency'].mean()
print("Mean crash rates:\n", mean_crash_rates)

Mean crash rates:
 IRI_Category
Good     34.007985
Fair    161.138578
Poor    414.560873
Name: Crash_Frequency, dtype: float64


/var/folders/yl/h3jt_tns2s14c8y5w9qptb6c0000gn/T/ipykernel_65825/3680048420.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IRI_Category'] = pd.cut(pd.to_numeric(df['IRI'], errors='coerce'), bins=iri_bins, labels=iri_labels)


In [30]:
#Fishers LSD Test 
import pandas as pd
import numpy as np
from scipy.stats import f, t

# Load your DataFrame
data= pd.read_csv("/Users/lebakuprathyushkumarreddy/Desktop/Prof_Lu_Gao/Crash_Data_(SOR)_2022_IRI_others.csv")
data["IRI"].fillna(df["IRI"].mean(), inplace=True)
data['Crash_Frequency'].fillna(df['Crash_Frequency'].mean(),inplace=True)

df = data[data["LIGHT"] == 1]


# Define your IRI bins and labels
iri_bins = [0, 95, 170, float('inf')]
iri_labels = ['Good', 'Fair', 'Poor']

# Create a new column 'IRI_Category' based on binning
df['IRI_Category'] = pd.cut(pd.to_numeric(df['IRI'], errors='coerce'), bins=iri_bins, labels=iri_labels)

# Perform ANOVA to test for overall group differences
groups = [df[df['IRI_Category'] == label]['Crash_Frequency'] for label in iri_labels]
n_groups = len(groups)
n_total = len(df['Crash_Frequency'])
n_obs = [len(group) for group in groups]
mean_group = [group.mean() for group in groups]
overall_mean = df['Crash_Frequency'].mean()

# Calculate the sum of squares between groups (SSB)
SSB = sum([n * (mean - overall_mean)**2 for n, mean in zip(n_obs, mean_group)])

# Calculate the mean square between groups (MSB)
MSB = SSB / (n_groups - 1)

# Calculate the mean square error (MSE)
SSE = sum([(n - 1) * group.var() for n, group in zip(n_obs, groups)])
MSE = SSE / (n_total - n_groups)

# Calculate the F-statistic
F = MSB / MSE

# Calculate the critical value from the F-distribution
alpha = 0.05
df1 = n_groups - 1
df2 = n_total - n_groups
critical_value = f.ppf(1 - alpha, df1, df2)

# Perform pairwise comparisons using Fisher's LSD and calculate p-values
from itertools import combinations
pairwise_comparisons = list(combinations(iri_labels, 2))
for label1, label2 in pairwise_comparisons:
    group1 = df[df['IRI_Category'] == label1]['Crash_Frequency']
    group2 = df[df['IRI_Category'] == label2]['Crash_Frequency']
    
    # Calculate the pooled standard error
    pooled_var = ((n_obs[0] - 1) * group1.var() + (n_obs[1] - 1) * group2.var()) / (n_obs[0] + n_obs[1] - 2)
    pooled_se = np.sqrt(pooled_var * (1/n_obs[0] + 1/n_obs[1]))
    
    # Calculate the LSD
    lsd = abs(mean_group[0] - mean_group[1]) / pooled_se
    # Calculate the critical LSD value
    critical_lsd = t.ppf(1 - alpha/2, df2) * pooled_se
   
    # Calculate the p-value
    p_value = 2 * (1 - t.cdf(abs(lsd), df2))
    
    if lsd > critical_lsd:
        print(f"Comparison between '{label1}' and '{label2}': LSD = {lsd:.4f} (significant), p-value = {p_value:.4f}")
        print(f"Value of LSD: {lsd}")
        print(f"Value of Crirical LSD: {critical_lsd}")

    else:
        print(f"Comparison between '{label1}' and '{label2}': LSD = {lsd:.4f} (not significant), p-value = {p_value:.4f}")
        print(f"Value of LSD: {lsd}")
        print(f"Value of Crirical LSD: {critical_lsd}")


Comparison between 'Good' and 'Fair': LSD = 66.5520 (significant), p-value = 0.0000
Value of LSD: 66.55200803081932
Value of Crirical LSD: 3.7441502377394067
Comparison between 'Good' and 'Poor': LSD = 9.5024 (not significant), p-value = 0.0000
Value of LSD: 9.502433284961862
Value of Crirical LSD: 26.222832533322766
Comparison between 'Fair' and 'Poor': LSD = 9.4988 (not significant), p-value = 0.0000
Value of LSD: 9.498800340680821
Value of Crirical LSD: 26.232861809238447


/var/folders/yl/h3jt_tns2s14c8y5w9qptb6c0000gn/T/ipykernel_65825/2933467852.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IRI_Category'] = pd.cut(pd.to_numeric(df['IRI'], errors='coerce'), bins=iri_bins, labels=iri_labels)


In [4]:
#Tukey Kramer Test
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Load your DataFrame
df = pd.read_csv("/Users/lebakuprathyushkumarreddy/Desktop/Prof_Lu_Gao/Crash_Data_(SOR)_2022_IRI_others.csv")
df["IRI"].fillna(df["IRI"].mean(), inplace=True)
df['Crash_Frequency'].fillna(df['Crash_Frequency'].mean(),inplace=True)

# Define your IRI bins and labels
iri_bins = [0, 95, 170, float('inf')]
iri_labels = ['Good', 'Fair', 'Poor']

# Create a new column 'IRI_Category' based on binning
df['IRI_Category'] = pd.cut(pd.to_numeric(df['IRI'], errors='coerce'), bins=iri_bins, labels=iri_labels)

# Perform Tukey-Kramer test
tukey_results = pairwise_tukeyhsd(endog=df['Crash_Frequency'], groups=df['IRI_Category'], alpha=0.05)

# Extract p-values from the summary
print(tukey_results.summary())




  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2  meandiff p-adj   lower     upper   reject
--------------------------------------------------------
  Fair   Good -123.1598   0.0 -129.8388 -116.4808   True
  Fair   Poor  256.4614   0.0   247.832  265.0909   True
  Good   Poor  379.6213   0.0  369.0889  390.1536   True
--------------------------------------------------------
